<a href="https://colab.research.google.com/github/aadityane93/SortingLine-ml/blob/main/wavelets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install tsfresh
import pywt

In [8]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
folder_id = '1ps_Bh93KQRJ4WbnilORmKd1BaWUXtmt1' # files

folder_list = drive.ListFile({'q': "'{}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false".format(folder_id)}).GetList()


for folder in folder_list:
    folder_title = folder['title']
    print(f"Processing folder: {folder_title}")
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder['id'])}).GetList()

    for index, file in enumerate(file_list):
        file_title = file['title']
        print(f"Downloading {file_title}")
        file.GetContentFile(file_title)

Processing folder: 400hz
Processing folder: 100hz
Processing folder: 800hz


In [23]:
import pandas as pd

In [27]:
train_target = pd.read_csv('100hz-train-target-50percent-10files.csv')
train_target.drop(columns=['Unnamed: 0'], inplace = True)
test_target = pd.read_csv('100hz-test-target-50percent-10files.csv')
test_target.drop(columns=['Unnamed: 0'], inplace = True)
train_feature = pd.read_csv('100hz-train-feature-50percent-10files.csv')
train_feature.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)', 'color'], inplace = True)
test_feature = pd.read_csv('100hz-test-feature-50percent-10files.csv')
test_feature.drop(columns=['Unnamed: 0', 'Datum', 'Energie_In(Wh)','Energie_Out(Wh)', 'P_In(W)', 'P_Out(W)', 'color' ], inplace = True)

In [28]:
train_feature

,Uhrzeit,I_In,V_In,I_Out(A),V_Out(V),Temp(°C),fileno
0,2024-05-02 11:19:09.845032:845,0.25,11.52,0.23,9.00,37.13,0
1,2024-05-02 11:19:09.853083:853,0.29,11.65,0.25,9.00,36.63,0
2,2024-05-02 11:19:09.861520:861,0.39,11.65,0.33,9.03,37.13,0
3,2024-05-02 11:19:09.873394:873,0.27,11.61,0.41,9.03,37.13,0
4,2024-05-02 11:19:09.881917:881,0.25,11.74,0.25,9.03,37.13,0
...,...,...,...,...,...,...,...
15127,2024-05-02 12:51:02.302558:302,0.31,11.70,0.29,9.03,37.88,34
15128,2024-05-02 12:51:02.310972:310,0.25,11.61,0.31,9.06,37.88,34
15129,2024-05-02 12:51:02.323006:323,0.25,11.70,0.25,9.03,37.88,34
15130,2024-05-02 12:51:02.331326:331,0.31,11.70,0.27,9.00,37.88,34


In [56]:
wavelet_type = 'db4'
decomp_level = 3

In [57]:
x_train_sorted = train_feature.sort_values(by=['fileno', 'Uhrzeit'])

# Group by fileno (each group is a time series)
grouped = x_train_sorted.groupby('fileno')

In [58]:
import numpy as np

In [63]:
wavelet_features = []
wavelet_df = {}

In [66]:

def tranform_ (column_):
  for fileno, group in grouped:
      signal = group[f'{column_}'].values  # Example: Use power input

      # Perform wavelet decomposition
      coeffs = pywt.wavedec(signal, wavelet_type, level=decomp_level)

      # Extract features from coefficients
      features = {}
      for i, coeff in enumerate(coeffs):
          if len(coeff) == 0:
              continue
          # features: energy, entropy, variance
          features[f'{column_}wavelet_energy_level_{i}'] = np.sum(coeff**2)
          features[f'{column_}wavelet_entropy_level_{i}'] = -np.sum(coeff**2 * np.log(coeff**2))
          features[f'{column_}wavelet_var_level_{i}'] = np.var(coeff)

      # Add fileno to map features back
      # features['fileno'] = fileno
      wavelet_features.append(features)
tranform_("I_In")
tranform_("I_Out(A)")
tranform_("V_In")
tranform_("V_Out(V)")

# Convert to DataFrame
# pywt.wavet features['fileno'] = fileno

wavelet_df = pd.DataFrame(wavelet_features).set_index('fileno')

In [65]:
wavelet_df

,I_Inwavelet_energy_level_0,I_Inwavelet_entropy_level_0,I_Inwavelet_var_level_0,I_Inwavelet_energy_level_1,I_Inwavelet_entropy_level_1,I_Inwavelet_var_level_1,I_Inwavelet_energy_level_2,I_Inwavelet_entropy_level_2,I_Inwavelet_var_level_2,I_Inwavelet_energy_level_3,...,V_Out(V)wavelet_var_level_0,V_Out(V)wavelet_energy_level_1,V_Out(V)wavelet_entropy_level_1,V_Out(V)wavelet_var_level_1,V_Out(V)wavelet_energy_level_2,V_Out(V)wavelet_entropy_level_2,V_Out(V)wavelet_var_level_2,V_Out(V)wavelet_energy_level_3,V_Out(V)wavelet_entropy_level_3,V_Out(V)wavelet_var_level_3
fileno,,,,,,,,,,,,,,,,,,,,,
0,66.471623,11.775565,0.022990,0.162372,0.580778,0.001885,0.297606,1.449194,0.001802,0.744620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,66.591676,12.826429,0.021254,0.620397,0.989104,0.007128,0.370664,1.524498,0.002222,0.526406,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,66.601118,13.474562,0.021062,0.230205,0.786372,0.002571,0.325732,1.605642,0.001948,0.611320,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64.307198,14.492834,0.017624,0.198601,0.746015,0.002327,0.259706,1.481864,0.001565,0.721140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,64.793704,11.847910,0.021779,0.266420,0.845199,0.003206,0.580471,1.685150,0.003632,0.535561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000430,0.051157,0.360658,0.000423,0.075999,0.544645,0.000326,0.164079,1.177752,0.000355
31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000349,0.038796,0.284651,0.000326,0.067481,0.501051,0.000290,0.147850,1.076020,0.000322
32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000360,0.037791,0.276873,0.000327,0.078678,0.565697,0.000352,0.168878,1.216830,0.000383
